# Livrable 1 - groupe 1

Nous sommes le groupe 1 composé de :  
- DELORME Alexandre
- ENCOGNERE Yanis
- MENNERON Laurine
- PEREON Alexandre
- ROCHARD Léo

## Contenu du livrable
Dans ce premier livrable, nous allons nous intéresser à la classification multiclasses de nos données. 
Nous avons à disposition : des peintures, des schémas/graphes, des portraits dessinés en noir et blanc, des images de textes scannés et des photos.  
Nous réaliserons dans un premier temps une analyse exploratoire des données (`EDA`) afin de mieux comprendre les caractéristiques de notre jeu de données, d’identifier d’éventuelles anomalies ou déséquilibres, et de préparer les données pour l’entraînement. Cette étape inclura notamment une visualisation des classes, la vérification des tailles et des formats d’images, ainsi que des traitements éventuels (redimensionnement, normalisation, data augmentation).  
Nous allons ensuite définir `l’architecture de notre modèle de classification d’images`. Nous justifierons les choix réalisés concernant la structure de notre réseau ainsi que les choix des `hyperparamètres` de nos différentes couches (nombre de filtres, taille des noyaux de convolution, fonctions d’activation, couches de pooling, etc.) et les paramètres d'entraînement (learning rate, batch size, nombre d’époques, fonction de perte, optimiseur, etc.).  
Nous entraînerons notre modèle en suivant une démarche itérative : 

__Entraînement :__  
Nous procéderons à l’entraînement du modèle sur notre jeu de données d’apprentissage en intégrant des `techniques de régularisation` (dropout, batch normalization, augmentation de données) pour limiter le sur-apprentissage. 

__Benchmark :__  
Nous mettrons en place un benchmark afin de comparer les performances de différentes architectures ou configurations de notre modèle. Nous testerons plusieurs variantes pour évaluer les impacts sur les performances. 

__Métriques :__  
Les performances du modèle seront évaluées à l’aide de plusieurs `métriques` : accuracy, matrice de confusion, précision, rappel et F1-score. Nous suivrons également la progression de la courbe d’apprentissage pour identifier les cas de sur et de sous-apprentissage. 

__Conclusion :__  
Enfin, nous conclurons ce livrable en récapitulant les résultats obtenus, les points forts et les limites de notre approche actuelle. Nous proposerons également des pistes d’amélioration possibles que nous pourrons appliquer pour les prochains livrables.

## Chargement des bibliothèques

Voici les bibliothèques principales que nous avons utilisées pour ce projet :  
- __NumPy :__ facilite et améliore la manipulation et le `traitement des données`
- __TensorFlow :__ bibliothèque principale de Deep Learning utilisée pour construire, entraîner et évaluer nos modèles de `classification`. Elle fournit des structures de données adaptées, des couches prédéfinies ainsi que les outils nécessaires pour la création de `réseaux de neurones` et leur entraînement
- __VisualKeras :__ outil permettant de générer des représentations graphiques de l’architecture de nos modèles Keras, facilitant ainsi leur visualisation et leur compréhension
- __Scikit-learn (sklearn) :__ utiliser pour `visualiser` et évaluer notre modèle. Utile également pour la réalisation de l’analyse `ACP` et `TSME`
- __Matplotlib et Seaborn :__ utilisation pour la visualisation de nos différents résultats sous forme de `graphiques`

In [ ]:
%load_ext tensorboard
import os
import pathlib
import zipfile
from datetime import datetime
from collections import defaultdict

import gdown
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import visualkeras
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model

## Sélection de la source de données


Cette partie permet de récuperer le dataset depuis le drive afin d'automatiser la `pipeline`.  
L'objectif est d'utiliser un dataset commun au sein du groupe. 

Ce dernier à le dossier `Sketch` complété par des données trouvés dans les sources suivantes : [croquis](https://paperswithcode.com/dataset/sketch) || [visages réalistes](https://www.kaggle.com/datasets/arbazkhan971/cuhk-face-sketch-database-cufs/data)

Ce qui fait passer le dossier sketch de `606 visages` et `800 croquis` avec un ration de `43%`/`57%` entre visages/croquis à `1200 visages` et `3200 croquis` pour un ratio final équivalent.

In [ ]:
# ID du fichier (extrait de l'URL)
file_id = "1d0k3mXd93JM0fLLYrUZRr4Un1F_TnuHi"
dataset_path = "dataset_livrable_1"
zip_path = dataset_path + ".zip"
extract_dir = pathlib.Path(zip_path).parent / dataset_path
reduce_dataset = True

if not os.path.exists(extract_dir):
    print(f"Le dossier '{extract_dir}' n'existe pas. Téléchargement en cours...")
    gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_path, quiet=False)

    print(f"Extraction ZIP en cours...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extraction Zip terminée")
else:
    print(f"Le dossier '{extract_dir}' existe déjà. Téléchargement et extraction non nécessaires.")

data_dir = extract_dir
print(f"Dataset disponible dans : {data_dir}")

## Détection des catégories

Voici ci-contre la liste des classes de notre dataset :

In [ ]:
categories = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
print(f"Catégories détectées : {categories}")

## Pré traitement des images

Certaines images de notre dataset sont `corrompues` ou ne sont pas au bon format (exemple avec un fichier « .ini »). Nous devons donc nous assurer de filtrer ces données avant de créer notre dataset. 

In [ ]:
# Suppresion des fichiers corrompus ou non images
def clean_images_dataset(dataset_path_arg):
    """
    Fonction pour nettoyer le dataset en supprimant les fichiers corrompus ou non images.
    """
    # Dictionnaire pour stocker le nombre d'images corrompues par classe
    corrupted_count_by_class = defaultdict(int)
    dataset_path = dataset_path_arg
    print("Début de la vérification des images ...")

    # Récupération de toutes les images pour calculer la progression
    all_files = []
    for dir_name in os.listdir(dataset_path):
        dir_path = os.path.join(dataset_path, dir_name)
        for file_name in os.listdir(dir_path):
            all_files.append((dir_name, dir_path, file_name))

    total_files = len(all_files)
    checked_files = 0  # Pour la progression

    # Parcours des images avec affichage de la progression
    for dir_name, dir_path, file_name in all_files:
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            try:
                with open(os.path.join(dir_path, file_name), 'rb') as file:
                    img_bytes = file.read()  # Lire les bytes de l'image
                    img = tf.image.decode_image(img_bytes)  # Essayer de décoder l'image
            except Exception as e:
                corrupted_count_by_class[dir_name] += 1
                print(f"\nImage corrompue : {file_name} dans {dir_name}. Exception: {e}")
                os.remove(os.path.join(dir_path, file_name))
                print(f"Image {file_name} supprimée.")
        else:
            corrupted_count_by_class[dir_name] += 1
            print(f"\nLe fichier {file_name} dans {dir_name} n'est pas une image.")
            os.remove(os.path.join(dir_path, file_name))
            print(f"Fichier {file_name} supprimé.")

        # Mise à jour de la progression
        checked_files += 1
        progress = (checked_files / total_files) * 100
        print(f"\rProgression : [{int(progress)}%] {checked_files}/{total_files} images vérifiées", end="")

    print("\nVérification des fichiers terminée.")

    # Affichage du nombre d'images corrompues par dossier
    for dir_name, count in corrupted_count_by_class.items():
        print(f"Dossier {dir_name} : {count} images corrompues")

    # Nombre total d'images corrompues
    total_corrupted = sum(corrupted_count_by_class.values())
    print(f"Nombre total d'images corrompues ou non image : {total_corrupted}")

## Calcul du nombre de données par classe

Nous comptons ensuite le nombre d’images par dossier pour pouvoir déterminer la `répartition des données par classe`.  
Nous observons que la classe Sketch est très largement `sous représentés` avec seulement `4394` données contre une moyenne de `9998` pour les autres classes.  
Pour pouvoir compenser ce déséquilibre, nous comptons sur la `loss pondération`. Ainsi, notre modèle se trouvera d’autant plus pénalisé si il effectue une mauvaise prédiction sur une image de classe Sketch.  
Nous avons également pensé à générer de nouvelles données à partir de `modèles génératifs` tel qu’un `GAN` et un `auto-encodeur de type VAE`. Cependant, les résultats que nous avons obtenus n’étant pas suffisamment satisfaisants, nous avons décidé de garder uniquement la loss pondération. 

In [ ]:
class_counts = {class_name: len(list((data_dir / class_name).glob('*'))) for class_name in categories}
class_counts['Sketch'] = len(list((data_dir / 'Sketch' / 'jpg').glob('*'))) + len(
    list((data_dir / 'Sketch' / 'png').glob('*')))
print(class_counts)

## Création du dataset

Nous pouvons finalement créer notre dataset à partir des images que nous avons traités précédemment. Nous avons défini un `batch de 32`, un batch élevé nous permet de gagner en performance lors de l'entraînement  au détriment de la mémoire, cette valeur est un bon compromis. Nous avons séparé nos données en 2 ensembles : `80%` pour l’entraînement et `20%` pour la validation.

In [ ]:
validation_split = 0.2
seed = 42
batch_size = 32
img_height = 180
img_width = 180

train_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    class_names=categories)

val_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    class_names=categories)

class_names = train_set.class_names
num_classes = len(class_names)

print(f"Classes found: {class_names}")

## Visualisation des différentes classes

Voici un échantillon d'images de notre dataset pour les différentes classes :

In [ ]:
plt.figure(figsize=(8, 8))
for images, labels in train_set.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

## Répartition des classes au sein du dataset

Le graphique ci-dessous représente la `répartition des données` pour nos datasets `d'entraînement` et de `validation` :

In [ ]:
# Compter les échantillons par classe dans train et val
def count_per_class(dataset, class_names):
    class_counts = defaultdict(int)
    for images, labels in dataset:
        for label in labels.numpy():
            class_counts[class_names[label]] += 1
    return class_counts

train_label_counts = count_per_class(train_set, class_names)
val_label_counts = count_per_class(val_set, class_names)

# S'assurer que toutes les classes sont présentes dans les deux sets
classes = class_names
train_counts = [train_label_counts.get(cls, 0) for cls in classes]
val_counts = [val_label_counts.get(cls, 0) for cls in classes]

x = np.arange(len(classes))  # position des classes

# Création du barplot
plt.figure(figsize=(12, 6))
bar_width = 0.35
plt.bar(x - bar_width/2, train_counts, width=bar_width, label='Train Set', color='skyblue')
plt.bar(x + bar_width/2, val_counts, width=bar_width, label='Validation Set', color='orange')

# Mise en forme
plt.xticks(x, classes, rotation=45)
plt.xlabel("Classes")
plt.ylabel("Nombre d'images")
plt.title("Répartition des images par classe (Train vs Validation)")
plt.legend()
plt.tight_layout()
plt.show()

## Analyse

### Préparation pour analyse


In [ ]:
# Convertir les images et labels en arrays
images, labels = zip(*[(image_batch.numpy(), label_batch.numpy()) for image_batch, label_batch in train_set])
images = np.concatenate(images)
labels = np.concatenate(labels)

# Sélectionner 20% des données aléatoirement
num_samples = images.shape[0]
sample_size = int(num_samples * 0.20)  # 20% du dataset
indices = np.random.choice(num_samples, sample_size, replace=False)

# Extraire les échantillons sélectionnés
images_sampled = images[indices]
labels_sampled = labels[indices]

### Analyse en composantes principales


In [ ]:
# Réduction dimensionnelle avec PCA
images_reshaped = images_sampled.reshape(images_sampled.shape[0], -1)
pca = PCA(n_components=0.95)
pca_result = pca.fit_transform(images_reshaped)

# Affichage PCA
plt.figure(figsize=(10, 8))
for i in range(len(class_names)):
    plt.scatter(pca_result[labels_sampled == i, 0], pca_result[labels_sampled == i, 1], label=class_names[i], alpha=0.4)
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("2D PCA Plot of Image Data")
plt.legend()
plt.show()

In [ ]:
x_min = np.round(pca_result[:, 0].min(), -4)
x_max = np.round(pca_result[:, 0].max(), -4)
y_min = np.round(pca_result[:, 1].min(), -4)
y_max = np.round(pca_result[:, 1].max(), -4)

# Créer des subplots avec 2 lignes et 3 colonnes (ajuster en fonction du nombre de classes)
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
axes = axes.flatten()  # Aplatir la matrice d'axes pour un accès facile

# Affichage de PCA pour chaque classe dans un subplot distinct
for i in range(len(class_names)):
    ax = axes[i]  # Sélectionner l'axe pour la classe i

    # Filtrer les points de la classe i
    ax.scatter(pca_result[labels_sampled == i, 0], pca_result[labels_sampled == i, 1],
               alpha=0.6, label=f"{class_names[i]}", color=f"C{i}")

    # Définir les mêmes limites globales pour tous les subplots
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    ax.set_xlabel("Principal Component 1")
    ax.set_ylabel("Principal Component 2")
    ax.set_title(f"PCA Scatter Plot for {class_names[i]}")
    ax.legend()

# Supprimer les axes inutilisés si le nombre de classes est inférieur au nombre de subplots
for j in range(len(class_names), len(axes)):
    fig.delaxes(axes[j])

# Ajuster l'espace entre les subplots pour éviter le chevauchement
plt.tight_layout()
plt.show()

### Interprétation des résultats

Ce graphique divise l'espace PCA en différentes sous-graphiques pour analyser la distribution individuelle de chaque catégorie.

Peinture (Painting) : Distribuée assez largement, avec des valeurs de PCA couvrant une large plage. Cela suggère une diversité importante dans les caractéristiques des peintures, rendant difficile une séparation nette.

Photo : Également bien dispersée, mais plus concentrée autour d'un noyau central, indiquant que les photos partagent plus de caractéristiques communes que les peintures.

Schéma (Schematics) : La distribution est plus linéaire, suggérant que les caractéristiques des schémas sont fortement corrélées à une dimension spécifique de la PCA.

Croquis (Sketch) : Cette catégorie semble former un cluster compact, mais décalé par rapport aux autres types d'images, suggérant qu'elle est relativement distincte dans l'espace des caractéristiques.

Texte : Très fortement regroupé dans une zone précise, ce qui confirme que les images de texte ont des caractéristiques très spécifiques et homogènes.

### Analyse TSNE


In [ ]:
# Réduction dimensionnelle avec t-SNE
tsne = TSNE(n_components=2, random_state=123)
tsne_result = tsne.fit_transform(images_reshaped)

# Affichage t-SNE
plt.figure(figsize=(10, 8))
for i in range(len(np.unique(labels_sampled))):
    plt.scatter(tsne_result[labels_sampled == i, 0], tsne_result[labels_sampled == i, 1], label=f"{class_names[i]}",
                alpha=0.4)
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.title("2D t-SNE Visualization of Image Data")
plt.legend()
plt.show()


In [ ]:
# Déterminer les limites globales des axes pour t-SNE
x_min, x_max = tsne_result[:, 0].min(), tsne_result[:, 0].max()
y_min, y_max = tsne_result[:, 1].min(), tsne_result[:, 1].max()

# Créer des subplots avec 2 lignes et 3 colonnes (tu peux ajuster en fonction du nombre de classes)
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
axes = axes.flatten()  # Aplatir pour faciliter l'accès aux axes

# Nombre de classes
num_classes = len(np.unique(labels_sampled))

# Affichage de t-SNE pour chaque classe dans un subplot distinct
for i in range(num_classes):
    ax = axes[i]  # Sélectionner l'axe pour la classe i

    # Filtrer les points de la classe i
    ax.scatter(tsne_result[labels_sampled == i, 0], tsne_result[labels_sampled == i, 1],
               alpha=0.6, label=f"{class_names[i]}", color=f"C{i}")

    # Définir les mêmes limites globales pour tous les subplots
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    ax.set_xlabel("t-SNE Component 1")
    ax.set_ylabel("t-SNE Component 2")
    ax.set_title(f"t-SNE for {class_names[i]}")
    ax.legend()

# Supprimer les axes inutilisés si le nombre de classes est inférieur au nombre de subplots
for j in range(num_classes, len(axes)):
    fig.delaxes(axes[j])

# Ajuster l'espace entre les subplots pour éviter le chevauchement
plt.tight_layout()
plt.show()

### Interprétation des résultats
Painting (bleu) et Photo (orange) : Très imbriquées, ce qui suggère que leurs caractéristiques sont proches dans l’espace des features. Cela peut s'expliquer par une similarité visuelle ou une proximité stylistique entre certains tableaux et photos réalistes.

Schematics (vert clair) : Bien qu’elles s’étendent sur une large zone, elles forment une structure allongée partiellement chevauchée avec les clusters Painting/Photo. Cela indique une diversité interne importante dans cette classe, mais aussi quelques similarités avec des photos/peintures (probablement des schémas techniques complexes).

Sketch (rouge) : Très bien séparées, formant un cluster compact et isolé en haut de la carte. Cela montre que les croquis possèdent des caractéristiques visuelles très distinctes (formes simples, contours marqués, peu de textures).

Text (violet) : Regroupé de manière très dense et distincte à droite, formant plusieurs sous-clusters. Cela peut indiquer différents styles de textes (manuscrits, imprimés, tapés), mais reste bien séparé des autres types d’images, ce qui est cohérent avec la nature très différente du contenu textuel.

## Modélisation

### Optimisation pour l'entrainement

Dans cette partie, vous devrez utiliser les fonctions [`Dataset.cache`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#cache) et [`Dataset.prefetch`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch) afin de configurer les données pour améliorer les performances de la façon suivante :
- `Dataset.cache()` : cette fonction sert à forcer le maintien des données en cache dans la mémoire. Vu que le réseau de neurones fait plusieurs passes (qu'on nomme _époque_ ou _epoch_ en anglais) sur les données durant l'apprentissage, cette fonction permet de ne pas avoir à recharger les images à chaque fois. 
- `Dataset.prefetch()` : cette fonction permet de faire le prétraitement de l'élément courant du jeu de données (par exemple le batch suivant) en même temps que l'entraînement/évaluation du batch courant par le modèle. Dans un environnement multi-processeurs ou multi-cœur, c'est un gain de temps non-négligeable.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
# Cette commande n'est pas nécessaire sur colab
train_set = train_set.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_set = val_set.prefetch(buffer_size=AUTOTUNE)

### Définition de la fonction de perte

Nous utilisons une `fonction de perte` adaptée à notre problème de classification multiclasses. Cette fonction permet de calculer l’erreur entre les prédictions du modèle et les valeurs réelles afin d’`ajuster les poids` du réseau lors de l’entraînement. 
Afin de limiter l’impact du déséquilibre des classes dans nos données, nous avons défini des poids spécifiques pour chaque classe (`class weights`), attribuant un poids plus élevé aux classes sous-représentées afin de `compenser` leur faible occurrence. 

In [ ]:
total = sum(class_counts.values())
weights = {class_name: (total / class_count) for class_name, class_count in class_counts.items()}

max_weight = max(weights.values())
class_weights = {class_name: weight / max_weight for class_name, weight in weights.items()}

print("Class Weights:", class_weights)

# Appliquer les poids dans la fonction de perte
class_weight_tensor = tf.constant([class_weights[class_name] for class_name in categories], dtype=tf.float32)

print("Class_weight_tensor:", class_weight_tensor)


@tf.keras.utils.register_keras_serializable()
def weighted_loss(y_true, y_pred):
    weights = tf.gather(class_weight_tensor, tf.cast(y_true, tf.int32))
    unweighted_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return unweighted_loss * weights

### Définition du modèle

#### Architecture
Nous avons construit un `réseau de neurones convolutionnels` (CNN), architecture particulièrement adaptée au traitement et à la `classification d’images`.  
Notre modèle suit une structure classique composée de plusieurs `couches` successives :  
- Couches de convolution : extraction des caractéristiques locales des images à l’aide de `filtres convolutionnels`
- Couches de pooling : `réduction de la dimensionnalité` et des paramètres, limitation du risque de sur-apprentissage
- Couches finales : `flatten pour vectoriser les données`, suivi de couches Dense pour la prise de décision. La dernière
- Couche Dense : utilise une `activation softmax` adaptée à la classification multiclasse

#### Techniques complémentaires
- Application de `Dropout` après certaines couches pour `limiter le sur-apprentissage` (overfitting) en désactivant aléatoirement des neurones lors de l’entraînement
- Choix du `nombre de couches`, du `nombre de neurones` et des `hyperparamètres` (kernel size, filtres, taux de dropout, etc.) basé sur :  
    * Nos recherches personnelles et expérimentations (tests itératifs, résultats obtenus en WKS)
    * Des comparaisons avec l’état de l’art sur des projets similaires
    * L’analyse des performances obtenues lors de nos différents essais (benchmark et validation croisée)

In [ ]:
model = tf.keras.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.1, seed=42),

    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.1, seed=42),

    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.1, seed=42),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(0.3, seed=42),
    tf.keras.layers.Dense(units=num_classes, activation='softmax'), ]
)

model.compile(optimizer='adam', loss=weighted_loss, metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

model.summary()

### Visualisation du modèle

Voici ci-contre la visualisation de notre modèle, cette dernière nous permet de représenter de manière claire l’architecture de notre réseau de neurones. 

In [ ]:
visualkeras.layered_view(model, legend=True)

### Entrainement

Nous pouvons finalement entraîner notre modèle.  
Nous définissons des `callbacks` :  
- Early stopping pour arrêter l’entraînement quand le modèle n’apprend plus (ou sur-apprend)
- Création de checkpoint pour sauvegarder notre modèle
- Nous avons défini les paramètres nécessaires : nombre d’epochs, dataset, callback, batch

#### Définition des callbacks

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='./L1_model.keras', save_best_only=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Fit

In [ ]:
epochs = 10

history = model.fit(train_set,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=val_set,
                    verbose=1,
                    callbacks=[early_stopping, model_checkpoint, tensorboard_callback])

## Courbe d’apprentissage

o Interprétation de la courbe :

- Surapprentissage : la situation de sur-apprentissage peut être détectée lorsque la courbe de loss de validation commence à augmenter par rapport à la loss d’entraînement qui est stabilisée. Pour lutter contre ce phénomène nous avons mis en place des couches de Dropout ainsi que la callback permettant le early-stopping. Ces actions préventives permette de d'arrêter l'entraînement avant de rentrer dans le cas échéant.

- Sous apprentissage : le modèle n’a pas encore extrait toutes les caractéristiques pertinentes des données. Les performances varient encore beaucoup d’une epoch à l’autre. Il convient d’ajouter des données ou des epochs pour améliorer les performances du modèle.Dans notre cas, nous n'avons pas eu ce problème car le dataset est correctement répartit avec une proportion de 80% de données d'entrainements pour 20% de données de validations sur un nombre total avoisinant les 45 000 données au total. 

- Validation vs training set : Comme évoqué ci-dessus, nous avons divisé notre jeu de données en 2 parties : une dédiée à l’entraînement de notre modèle, et l’autre dédiée à la validation de son apprentissage. Ainsi nous vérifions sa capacité de généralisation sur des nouvelles données.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model = load_model('L1_model.keras')

## Métriques

### Matrice de confusion et rapport de classification

In [ ]:
def confusion_matrix_and_report(model, val_set, class_names):
    print("Classes utilisées :", class_names)

    # Préparer les données de validation sans modification
    val_set = val_set.prefetch(buffer_size=tf.data.AUTOTUNE)

    # Obtenir les vraies étiquettes et les prédictions
    val_true_labels = []
    val_pred_labels = []

    for images, labels in val_set:
        # Prédictions pour le batch
        predictions = model.predict(images, verbose=0)
        val_pred_labels.extend(np.argmax(predictions, axis=1))
        val_true_labels.extend(labels.numpy())

    # Convertir en arrays numpy
    val_true_labels = np.array(val_true_labels)
    val_pred_labels = np.array(val_pred_labels)

    # Calculer la matrice de confusion
    conf_matrix = confusion_matrix(val_true_labels, val_pred_labels)

    # Afficher la matrice de confusion
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
                xticklabels=class_names, yticklabels=class_names)
    plt.title("Matrice de Confusion")
    plt.xlabel("Classe Prédite")
    plt.ylabel("Classe Réelle")
    plt.show()

    # Afficher le rapport de classification
    report = classification_report(val_true_labels, val_pred_labels, target_names=class_names)
    print("Rapport de Classification :")
    print(report)

In [ ]:
confusion_matrix_and_report(model, val_set, class_names)

## Matrice de confusion

La matrice de confusion permet de comparer la classe réelle et la classe prédite pour les données de validation envoyées au modèle. Cette matrice nous permet de mieux comprendre les erreurs de notre modèle afin de pouvoir s’adapter en conséquence. En l’occurrence, nous constatons que notre modèle a parfois du mal à distinguer les photos des peintures (qui peuvent avoir un rendu assez réaliste). Ainsi nous pouvons proposer des pistes d’amélioration telles que l’augmentation de la qualité de l’image entrante par exemple.

Precision, Recall, F1score : Ces métriques permettent de mieux analyser les performances du modèle, et donc de s’assurer qu’il n’y a pas trop de confusions entre les différentes classes.

### Résultats globaux
- **Accuracy** : **85%**
- **Macro Average** (moyenne non pondérée) :
  - Précision : **0.87**
  - Rappel : **0.87**
  - F1-score : **0.86**
- **Weighted Average** (pondérée par le nombre d'exemples) :
  - Précision : **0.85**
  - Rappel : **0.85**
  - F1-score : **0.84**
### Forces
- **Sketch** et **Text** sont très bien reconnus, avec des F1-scores proches de 1.
- **Schematics** présente un bon équilibre entre précision et rappel.
### Faiblesses
- **Photo** :
  - Faible rappel (**0.62**) → le modèle rate de nombreux vrais exemples de "Photo".
- **Painting** :
  - Faible précision (**0.70**) → le modèle confond "Painting" avec des "Photos" et des "Schematics".

## Benchmarking
TODO

In [ ]:
%tensorboard --logdir logs/fit

## Conclusion

TODO :
- Rappel des objectives
- Résultats obtenues

OUVERTURE :
- Data augmentation : la data augmentation permet de rendre notre modèle plus robuste 
- Utilisation de transfert learning avec un modèle pré-entraîné (VGG16, ResNet, etc.)  
- Autoencoder livrable 2 => réduire la taille des données avec l’encodeur et utilisation du décodeur pour enlever le bruit
- Modèle BIAIS dans les couches 